<a href="https://colab.research.google.com/github/samnatale3/Data-Science-In-Football/blob/main/data_science_as2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


Mount the drive and display the raw data

In [ ]:
drive.mount('/content/gdrive')

# List of CSV files in the 'data_science_as2' folder
csv_files = [
    '/content/gdrive/MyDrive/data_science_as2/2017.csv',
    '/content/gdrive/MyDrive/data_science_as2/2018.csv',
    '/content/gdrive/MyDrive/data_science_as2/2019.csv',
    '/content/gdrive/MyDrive/data_science_as2/2020.csv',
    '/content/gdrive/MyDrive/data_science_as2/2021.csv'
]

# Load and concatenate the datasets
dataframes = []

# Set the max_columns display option
pd.set_option('display.max_columns', None)

for file in csv_files:
    temp_df = pd.read_csv(file)
    temp_df['Year'] = int(file[-8:-4])
    dataframes.append(temp_df)

data = pd.concat(dataframes, ignore_index=True)

# Display the raw data
data.head()

Mounted at /content/gdrive


,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches,-9999\,Year
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,26.0,1990.0,28,25,2184,24.3,5,1,6,5,0,0,7,0,3.1,3.1,2.1,5.2,46.0,92.0,86.0,0.21,0.04,0.25,0.21,0.25,0.13,0.09,0.21,0.13,0.21,Matches,5f09991f\,2017
1,Rolando Aarons,eng ENG,MFFW,Newcastle Utd,eng Premier League,21.0,1995.0,4,1,139,1.5,0,0,0,0,0,0,0,0,0.1,0.1,0.0,0.1,7.0,3.0,4.0,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.04,0.04,0.04,Matches,c5942695\,2017
2,Rolando Aarons,eng ENG,MFFW,Hellas Verona,it Serie A,21.0,1995.0,11,6,517,5.7,0,0,0,0,0,0,0,0,0.2,0.2,0.2,0.3,9.0,17.0,31.0,0.00,0.00,0.00,0.00,0.00,0.03,0.03,0.06,0.03,0.06,Matches,c5942695\,2017
3,Ignazio Abate,it ITA,DF,Milan,it Serie A,30.0,1986.0,17,11,1057,11.7,1,0,1,1,0,0,3,0,0.2,0.2,0.5,0.7,20.0,81.0,65.0,0.09,0.00,0.09,0.09,0.09,0.01,0.04,0.06,0.01,0.06,Matches,1c529186\,2017
4,Aymen Abdennour,tn TUN,DF,Marseille,fr Ligue 1,27.0,1989.0,8,6,499,5.5,0,0,0,0,0,0,3,0,0.1,0.1,0.0,0.1,2.0,12.0,0.0,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.02,Matches,2f798b41\,2017


In [ ]:
# Assuming 'data' is the DataFrame containing the raw data
data.columns = ['Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls_per90', 'Ast_per90', 'G+A_per90', 'G-PK_per90', 'G+A-PK_per90', 'xG_per90', 'xAG_per90', 'xG+xAG_per90', 'npxG_per90', 'npxG+xAG_per90', 'Matches', '-9999', 'Year']

# Display the cleaned data
data.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls_per90,Ast_per90,G+A_per90,G-PK_per90,G+A-PK_per90,xG_per90,xAG_per90,xG+xAG_per90,npxG_per90,npxG+xAG_per90,Matches,-9999,Year
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,26.0,1990.0,28,25,2184,24.3,5,1,6,5,0,0,7,0,3.1,3.1,2.1,5.2,46.0,92.0,86.0,0.21,0.04,0.25,0.21,0.25,0.13,0.09,0.21,0.13,0.21,Matches,5f09991f\,2017
1,Rolando Aarons,eng ENG,MFFW,Newcastle Utd,eng Premier League,21.0,1995.0,4,1,139,1.5,0,0,0,0,0,0,0,0,0.1,0.1,0.0,0.1,7.0,3.0,4.0,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.04,0.04,0.04,Matches,c5942695\,2017
2,Rolando Aarons,eng ENG,MFFW,Hellas Verona,it Serie A,21.0,1995.0,11,6,517,5.7,0,0,0,0,0,0,0,0,0.2,0.2,0.2,0.3,9.0,17.0,31.0,0.00,0.00,0.00,0.00,0.00,0.03,0.03,0.06,0.03,0.06,Matches,c5942695\,2017
3,Ignazio Abate,it ITA,DF,Milan,it Serie A,30.0,1986.0,17,11,1057,11.7,1,0,1,1,0,0,3,0,0.2,0.2,0.5,0.7,20.0,81.0,65.0,0.09,0.00,0.09,0.09,0.09,0.01,0.04,0.06,0.01,0.06,Matches,1c529186\,2017
4,Aymen Abdennour,tn TUN,DF,Marseille,fr Ligue 1,27.0,1989.0,8,6,499,5.5,0,0,0,0,0,0,3,0,0.1,0.1,0.0,0.1,2.0,12.0,0.0,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.02,Matches,2f798b41\,2017


Remove unnessasary columns and seperate the player by year, additionally if a player has played for two teams in one year then join the squad name so they don't have duplicate records

In [ ]:
# Group by Player, Nation, Pos, Age, Born, and Year, then aggregate the statistics
grouped_data = data.groupby(['Player', 'Pos', 'Nation', 'Age', 'Born', 'Year'], as_index=False).agg({

    'Squad': ', '.join,
    'Comp': ', '.join,
    'MP': 'sum',
    'Starts': 'sum',
    'Min': 'sum',
    '90s': 'sum',
    'Gls': 'sum',
    'Ast': 'sum',
    'G+A': 'sum',
    'G-PK': 'sum',
    'PK': 'sum',
    'PKatt': 'sum',
    'CrdY': 'sum',
    'CrdR': 'sum',
    'xG': 'sum',
    'npxG': 'sum',
    'xAG': 'sum',
    'npxG+xAG': 'sum',
    'PrgC': 'sum',
    'PrgP': 'sum',
    'PrgR': 'sum'
})

# Display the cleaned data
grouped_data.head()

,Player,Pos,Nation,Age,Born,Year,Squad,Comp,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR
0,A\'efssa Mandi,DF,dz ALG,25.0,1991.0,2017,Betis,es La Liga,34,34,3024,33.6,1,0,1,1,0,0,9,1,3.3,3.3,0.2,3.5,30.0,129.0,9.0
1,A\'efssa Mandi,DF,dz ALG,26.0,1991.0,2018,Betis,es La Liga,35,34,3059,34.0,1,1,2,1,0,0,10,0,0.6,0.6,1.5,2.2,23.0,172.0,17.0
2,A\'efssa Mandi,DF,dz ALG,27.0,1991.0,2019,Betis,es La Liga,29,29,2576,28.6,0,0,0,0,0,0,9,0,1.5,1.5,1.1,2.6,10.0,93.0,8.0
3,A\'efssa Mandi,DF,dz ALG,28.0,1991.0,2020,Betis,es La Liga,28,27,2392,26.6,3,1,4,3,0,0,6,2,2.1,2.1,0.8,2.9,5.0,62.0,3.0
4,A\'efssa Mandi,DF,dz ALG,29.0,1991.0,2021,Villarreal,es La Liga,17,14,1375,15.3,1,0,1,1,0,0,3,0,2.3,2.3,0.1,2.3,7.0,54.0,2.0


Display null values so they can be addressed

In [ ]:
null_counts = grouped_data.isnull().sum()
print(null_counts)

total_nulls = null_counts.sum()
print("Total null values in the dataset:", total_nulls)

Player      0
Pos         0
Nation      0
Age         0
Born        0
Year        0
Squad       0
Comp        0
MP          0
Starts      0
Min         0
90s         0
Gls         0
Ast         0
G+A         0
G-PK        0
PK          0
PKatt       0
CrdY        0
CrdR        0
xG          0
npxG        0
xAG         0
npxG+xAG    0
PrgC        0
PrgP        0
PrgR        0
dtype: int64
Total null values in the dataset: 0


Normalise the data by cleaning it's datatypes and preprocessing it ready for training

In [ ]:
# List of columns to clean
cols_to_clean = ['Player', 'Nation', 'Squad', 'Comp']

# Remove special characters, whitespace, and punctuation, and convert to lowercase
for col in cols_to_clean:
    grouped_data[col] = grouped_data[col].str.replace('[^\w\s]', '')  # Remove punctuation
    grouped_data[col] = grouped_data[col].str.strip()  # Remove whitespace
    grouped_data[col] = grouped_data[col].str.lower()  # Convert to lowercase

grouped_data['Age'] = grouped_data['Age'].astype(int)
grouped_data['Born'] = grouped_data['Born'].astype(int)

# Display the cleaned data
grouped_data.head()


<ipython-input-6-85cc8fd581bc>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  grouped_data[col] = grouped_data[col].str.replace('[^\w\s]', '')  # Remove punctuation


,Player,Pos,Nation,Age,Born,Year,Squad,Comp,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR
0,aefssa mandi,DF,dz alg,25,1991,2017,betis,es la liga,34,34,3024,33.6,1,0,1,1,0,0,9,1,3.3,3.3,0.2,3.5,30.0,129.0,9.0
1,aefssa mandi,DF,dz alg,26,1991,2018,betis,es la liga,35,34,3059,34.0,1,1,2,1,0,0,10,0,0.6,0.6,1.5,2.2,23.0,172.0,17.0
2,aefssa mandi,DF,dz alg,27,1991,2019,betis,es la liga,29,29,2576,28.6,0,0,0,0,0,0,9,0,1.5,1.5,1.1,2.6,10.0,93.0,8.0
3,aefssa mandi,DF,dz alg,28,1991,2020,betis,es la liga,28,27,2392,26.6,3,1,4,3,0,0,6,2,2.1,2.1,0.8,2.9,5.0,62.0,3.0
4,aefssa mandi,DF,dz alg,29,1991,2021,villarreal,es la liga,17,14,1375,15.3,1,0,1,1,0,0,3,0,2.3,2.3,0.1,2.3,7.0,54.0,2.0


Iterates through the column names in numeric_cols, and for each column, it applies the pd.to_numeric() function to convert the data type of the column to a numeric data type (integer or float).

In [ ]:
numeric_cols = ['Age', 'Born', 'Year', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR']
for col in numeric_cols:
    grouped_data[col] = pd.to_numeric(grouped_data[col], errors='coerce')

# Display the cleaned data
grouped_data.head()

,Player,Pos,Nation,Age,Born,Year,Squad,Comp,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR
0,aefssa mandi,DF,dz alg,25,1991,2017,betis,es la liga,34,34,3024,33.6,1,0,1,1,0,0,9,1,3.3,3.3,0.2,3.5,30.0,129.0,9.0
1,aefssa mandi,DF,dz alg,26,1991,2018,betis,es la liga,35,34,3059,34.0,1,1,2,1,0,0,10,0,0.6,0.6,1.5,2.2,23.0,172.0,17.0
2,aefssa mandi,DF,dz alg,27,1991,2019,betis,es la liga,29,29,2576,28.6,0,0,0,0,0,0,9,0,1.5,1.5,1.1,2.6,10.0,93.0,8.0
3,aefssa mandi,DF,dz alg,28,1991,2020,betis,es la liga,28,27,2392,26.6,3,1,4,3,0,0,6,2,2.1,2.1,0.8,2.9,5.0,62.0,3.0
4,aefssa mandi,DF,dz alg,29,1991,2021,villarreal,es la liga,17,14,1375,15.3,1,0,1,1,0,0,3,0,2.3,2.3,0.1,2.3,7.0,54.0,2.0


Display possible positions

In [ ]:
print(grouped_data['Pos'].unique())

['DF' 'GK' 'DFFW' 'FW' 'FWMF' 'MFFW' 'MF' 'DFMF' 'MFDF' 'FWDF']


Run a basic training model to get some values

In [ ]:
train_data = grouped_data[(grouped_data['Year'] >= 2017) & (grouped_data['Year'] <= 2020) & (grouped_data['Pos'].str.contains('FW', case=False))]

train_data['Avg_Perf'] = (train_data['Gls'] + train_data['Ast']) / train_data['90s']

features = numeric_cols
X_train = train_data[features]
y_train = train_data['Gls']

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
model.fit(X_train, y_train)

test_data = grouped_data[(grouped_data['Year'] == 2021) & (grouped_data['Pos'].str.contains('FW', case=False))]

X_test = test_data[features]

test_data['Predicted_Gls'] = model.predict(X_test)

top_performers = test_data.sort_values(by='Predicted_Gls', ascending=False).head(10)
print(top_performers[['Player', 'Nation', 'Squad', 'Age', 'Born', 'Year', 'Predicted_Gls']])

<ipython-input-9-05480cde36d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Avg_Perf'] = (train_data['Gls'] + train_data['Ast']) / train_data['90s']


                      Player  Nation                Squad  Age  Born  Year  \
10813     robert lewandowski  pl pol        bayern munich   32  1988  2021   
7066          kylian mbappe9  fr fra             paris sg   22  1998  2021   
6655           karim benzema  fr fra          real madrid   33  1987  2021   
2232           ciro immobile  it ita                lazio   31  1990  2021   
12946      wissam ben yedder  fr fra               monaco   30  1990  2021   
3175   du9aan vlahoviuc0u263  rs srb  fiorentina juventus   21  2000  2021   
10038          patrik schick  cz cze           leverkusen   25  1996  2021   
9110           mohamed salah  eg egy            liverpool   29  1992  2021   
11825           son heungmin  kr kor            tottenham   29  1992  2021   
3526          erling haaland  no nor             dortmund   21  2000  2021   

       Predicted_Gls  
10813      35.600570  
7066       27.963361  
6655       27.008340  
2232       26.996269  
12946      25.065902  
317

<ipython-input-9-05480cde36d2>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Predicted_Gls'] = model.predict(X_test)


Run a more advanced model that also contains assists to predict goal involvements rather than just goals, additionally shows mae, mse, rmse and r2

In [ ]:
train_data = grouped_data[(grouped_data['Year'] >= 2017) & (grouped_data['Year'] <= 2020) & (grouped_data['Pos'].str.contains('FW', case=False))]

train_data['Avg_Perf'] = (train_data['Gls'] + train_data['Ast']) / train_data['90s']

features = numeric_cols
X_train = train_data[features]

# Train a model for predicting goals
y_train_goals = train_data['Gls']
model_goals = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
model_goals.fit(X_train, y_train_goals)

# Train a model for predicting assists
y_train_assists = train_data['Ast']
model_assists = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
model_assists.fit(X_train, y_train_assists)

test_data = grouped_data[(grouped_data['Year'] == 2021) & (grouped_data['Pos'].str.contains('FW', case=False))]
X_test = test_data[features]

test_data['Predicted_Gls'] = model_goals.predict(X_test)
test_data['Predicted_Ast'] = model_assists.predict(X_test)

# Calculate the predicted goal involvements
test_data['Predicted_GI'] = test_data['Predicted_Gls'] + test_data['Predicted_Ast']

# Sort by predicted goal involvements and display the top 10 players
top_performers = test_data.sort_values(by='Predicted_GI', ascending=False)
print(top_performers[['Player', 'Nation', 'Squad', 'Age', 'Born', 'Year', 'Predicted_Gls', 'Predicted_Ast', 'Predicted_GI']])
print()
# Assuming you have the actual goals and assists for the test data
y_test_goals = test_data['Gls']
y_test_assists = test_data['Ast']

# Predict the goals and assists for the test data
y_test_goals_pred = test_data['Predicted_Gls']
y_test_assists_pred = test_data['Predicted_Ast']

# Calculate the evaluation metrics for goals predictions
mae_goals = mean_absolute_error(y_test_goals, y_test_goals_pred)
mse_goals = mean_squared_error(y_test_goals, y_test_goals_pred)
rmse_goals = np.sqrt(mse_goals)
r2_goals = r2_score(y_test_goals, y_test_goals_pred)

# Calculate the evaluation metrics for assists predictions
mae_assists = mean_absolute_error(y_test_assists, y_test_assists_pred)
mse_assists = mean_squared_error(y_test_assists, y_test_assists_pred)
rmse_assists = np.sqrt(mse_assists)
r2_assists = r2_score(y_test_assists, y_test_assists_pred)

print(f"Goals predictions - MAE: {mae_goals}, MSE: {mse_goals}, RMSE: {rmse_goals}, R2: {r2_goals}")
print(f"Assists predictions - MAE: {mae_assists}, MSE: {mse_assists}, RMSE: {rmse_assists}, R2: {r2_assists}")
print()

<ipython-input-10-82951d7fee1f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Avg_Perf'] = (train_data['Gls'] + train_data['Ast']) / train_data['90s']


                   Player   Nation           Squad  Age  Born  Year  \
7066       kylian mbappe9   fr fra        paris sg   22  1998  2021   
6655        karim benzema   fr fra     real madrid   33  1987  2021   
10813  robert lewandowski   pl pol   bayern munich   32  1988  2021   
9110        mohamed salah   eg egy       liverpool   29  1992  2021   
2199   christopher nkunku   fr fra      rb leipzig   23  1997  2021   
...                   ...      ...             ...  ...   ...   ...   
5793    jesuran rak sakyi  eng eng  crystal palace   18  2002  2021   
3024         dilane bakwa   fr fra        bordeaux   18  2002  2021   
1789                bryan   es esp       tottenham   20  2001  2021   
4370       germe1n valera   es esp   real sociedad   19  2002  2021   
13029         yanis lhery   fr fra   saintc9tienne   18  2003  2021   

       Predicted_Gls  Predicted_Ast  Predicted_GI  
7066       27.963361      16.038367     44.001728  
6655       27.008340      11.999695     39.

<ipython-input-10-82951d7fee1f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Predicted_Gls'] = model_goals.predict(X_test)
<ipython-input-10-82951d7fee1f>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Predicted_Ast'] = model_assists.predict(X_test)
<ipython-input-10-82951d7fee1f>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

Using gridsearch find the best possible parameters for training the model to achieve the best accuracy

In [ ]:
# Define the hyperparameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8, 0.9, 1.0],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3]
}

# Create a GradientBoostingRegressor model
model = GradientBoostingRegressor(random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train_goals)

# Find the best hyperparameters
best_params = grid_search.best_params_
print(f"Best hyperparameters: {best_params}")

# Train the model with the best hyperparameters
best_model = GradientBoostingRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train_goals)


Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 200, 'subsample': 0.9}


GradientBoostingRegressor(max_depth=4, min_samples_split=4, n_estimators=200,
                          random_state=42, subsample=0.9)

Reevaluate train the model using the values gained from above

In [ ]:
train_data = grouped_data[(grouped_data['Year'] >= 2017) & (grouped_data['Year'] <= 2020) & (grouped_data['Pos'].str.contains('FW', case=False))]

train_data['Avg_Perf'] = (train_data['Gls'] + train_data['Ast']) / train_data['90s']

features = numeric_cols
X_train = train_data[features]

# Train a model for predicting goals
y_train_goals = train_data['Gls']
model_goals = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42, subsample=0.9, min_samples_leaf=1, min_samples_split=4)
model_goals.fit(X_train, y_train_goals)

# Train a model for predicting assists
y_train_assists = train_data['Ast']
model_assists = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42, subsample=0.9, min_samples_leaf=1, min_samples_split=4)
model_assists.fit(X_train, y_train_assists)

test_data = grouped_data[(grouped_data['Year'] == 2021) & (grouped_data['Pos'].str.contains('FW', case=False))]
X_test = test_data[features]

test_data['Predicted_Gls'] = model_goals.predict(X_test)
test_data['Predicted_Ast'] = model_assists.predict(X_test)

# Calculate the predicted goal involvements
test_data['Predicted_GI'] = test_data['Predicted_Gls'] + test_data['Predicted_Ast']

# Sort by predicted goal involvements and display the top 10 players
top_performers = test_data.sort_values(by='Predicted_GI', ascending=False)
print(top_performers[['Player', 'Nation', 'Squad', 'Age', 'Born', 'Year', 'Predicted_Gls', 'Predicted_Ast', 'Predicted_GI']])
print()

# Assuming you have the actual goals and assists for the test data
y_test_goals = test_data['Gls']
y_test_assists = test_data['Ast']

# Predict the goals and assists for the test data
y_test_goals_pred = test_data['Predicted_Gls']
y_test_assists_pred = test_data['Predicted_Ast']

# Calculate the evaluation metrics for goals predictions
mae_goals = mean_absolute_error(y_test_goals, y_test_goals_pred)
mse_goals = mean_squared_error(y_test_goals, y_test_goals_pred)
rmse_goals = np.sqrt(mse_goals)
r2_goals = r2_score(y_test_goals, y_test_goals_pred)

# Calculate the evaluation metrics for assists predictions
mae_assists = mean_absolute_error(y_test_assists, y_test_assists_pred)
mse_assists = mean_squared_error(y_test_assists, y_test_assists_pred)
rmse_assists = np.sqrt(mse_assists)
r2_assists = r2_score(y_test_assists, y_test_assists_pred)

print(f"Goals predictions - MAE: {mae_goals}, MSE: {mse_goals}, RMSE: {rmse_goals}, R2: {r2_goals}")
print(f"Assists predictions - MAE: {mae_assists}, MSE: {mse_assists}, RMSE: {rmse_assists}, R2: {r2_assists}")
print()

<ipython-input-12-dd0a66b54803>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Avg_Perf'] = (train_data['Gls'] + train_data['Ast']) / train_data['90s']


                   Player   Nation          Squad  Age  Born  Year  \
7066       kylian mbappe9   fr fra       paris sg   22  1998  2021   
6655        karim benzema   fr fra    real madrid   33  1987  2021   
10813  robert lewandowski   pl pol  bayern munich   32  1988  2021   
9110        mohamed salah   eg egy      liverpool   29  1992  2021   
2199   christopher nkunku   fr fra     rb leipzig   23  1997  2021   
...                   ...      ...            ...  ...   ...   ...   
8007      marcel sabitzer   at aut     rb leipzig   27  1994  2021   
2533       daniel ginczek   de ger      wolfsburg   30  1991  2021   
5926      joel pohjanpalo   fi fin     leverkusen   26  1994  2021   
12488         troy deeney  eng eng        watford   33  1988  2021   
5223        je9re9my pied   fr fra          lille   32  1989  2021   

       Predicted_Gls  Predicted_Ast  Predicted_GI  
7066       28.285090   1.620677e+01     44.491861  
6655       27.177154   1.199999e+01     39.177143  
108

<ipython-input-12-dd0a66b54803>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Predicted_Gls'] = model_goals.predict(X_test)
<ipython-input-12-dd0a66b54803>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Predicted_Ast'] = model_assists.predict(X_test)
<ipython-input-12-dd0a66b54803>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

K-fold cross-validation: This technique involves dividing the dataset into k smaller subsets, training the model on k-1 subsets, and validating the model on the remaining subset. 

This process is repeated k times, with each subset used for validation once. The average performance metrics across the k iterations can give us a better understanding of the model's performance as it seems very accurate based on it's scores previously

In [ ]:
train_data = grouped_data[(grouped_data['Year'] >= 2017) & (grouped_data['Year'] <= 2020) & (grouped_data['Pos'].str.contains('FW', case=False))]
features = numeric_cols
X_train = train_data[features]
y_train_goals = train_data['Gls']
y_train_assists = train_data['Ast']

# Define the models with the hyperparameters you've set previously
model_goals = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42, subsample=0.9, min_samples_leaf=1, min_samples_split=4)
model_assists = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42, subsample=0.9, min_samples_leaf=1, min_samples_split=4)

# Perform k-fold cross-validation (e.g., with k=5)
k = 5
kf = KFold(n_splits=k, random_state=42, shuffle=True)

# Calculate the cross-validated performance metrics for goals and assists predictions
mae_goals_cv = -1 * cross_val_score(model_goals, X_train, y_train_goals, cv=kf, scoring='neg_mean_absolute_error').mean()
mse_goals_cv = -1 * cross_val_score(model_goals, X_train, y_train_goals, cv=kf, scoring='neg_mean_squared_error').mean()
rmse_goals_cv = np.sqrt(mse_goals_cv)
r2_goals_cv = cross_val_score(model_goals, X_train, y_train_goals, cv=kf, scoring='r2').mean()

mae_assists_cv = -1 * cross_val_score(model_assists, X_train, y_train_assists, cv=kf, scoring='neg_mean_absolute_error').mean()
mse_assists_cv = -1 * cross_val_score(model_assists, X_train, y_train_assists, cv=kf, scoring='neg_mean_squared_error').mean()
rmse_assists_cv = np.sqrt(mse_assists_cv)
r2_assists_cv = cross_val_score(model_assists, X_train, y_train_assists, cv=kf, scoring='r2').mean()

print(f"Goals predictions (cross-validated) - MAE: {mae_goals_cv}, MSE: {mse_goals_cv}, RMSE: {rmse_goals_cv}, R2: {r2_goals_cv}")
print(f"Assists predictions (cross-validated) - MAE: {mae_assists_cv}, MSE: {mse_assists_cv}, RMSE: {rmse_assists_cv}, R2: {r2_assists_cv}")


Goals predictions (cross-validated) - MAE: 0.0053391228405649145, MSE: 0.016382525788234646, RMSE: 0.12799424123074696, R2: 0.9992877746650664
Assists predictions (cross-validated) - MAE: 0.002121273788525936, MSE: 0.004765118887013567, RMSE: 0.06902984055474536, R2: 0.999197494095126


More refined for finding young players that would be good to purchase due to their high predicted goal involvements

In [ ]:
young_prospects = test_data[test_data['Age'] < 25]
young_prospects = young_prospects.sort_values(by='Predicted_GI', ascending=False)
print(young_prospects.head(20))

                      Player   Pos   Nation  Age  Born  Year  \
7066          kylian mbappe9    FW   fr fra   22  1998  2021   
2199      christopher nkunku  FWMF   fr fra   23  1997  2021   
3526          erling haaland    FW   no nor   21  2000  2021   
12732       vinicius jfanior    FW   br bra   21  2000  2021   
3175   du9aan vlahoviuc0u263    FW   rs srb   21  2000  2021   
9188            moussa diaby  FWMF   fr fra   22  1999  2021   
7137       lautaro martednez    FW   ar arg   23  1997  2021   
8372          martin terrier  FWMF   fr fra   24  1997  2021   
5487            jarrod bowen    FW  eng eng   24  1996  2021   
12098          tammy abraham    FW  eng eng   23  1997  2021   
10518           rafael lee3o    FW   pt por   22  1999  2021   
5386          james maddison  MFFW  eng eng   24  1996  2021   
3062              diogo jota    FW   pt por   24  1996  2021   
718             amine gouiri  FWMF   fr fra   21  2000  2021   
9104     mohamed lamine bayo    FW   gn 

A new model for finding talented midfielders using multiple datafields to give the player an overall predicted score, higher the score better the performance.

Key focus being:

*   Predicted goal involvements
*   Predicted progressive carries (carrying the ball up the pitch into more favourable position)
*   Predicted progressive passes (passes further up the pitch, leading to goal scoring opportunities)
*   Predicted progressive runs (runs that can lead to goal scoring opportunities)





In [ ]:
train_data = grouped_data[(grouped_data['Year'] >= 2017) & (grouped_data['Year'] <= 2020) & (grouped_data['Pos'].str.contains('MF', case=False))]

features = numeric_cols
X_train = train_data[features]

# Train a model for each metric
metrics = ['GI', 'PrgC', 'PrgP', 'PrgR']
models = {}
for metric in metrics:
    train_data[metric] = train_data['Gls'] + train_data['Ast'] if metric == 'GI' else train_data[metric]
    y_train = train_data[metric]
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
    model.fit(X_train, y_train)
    models[metric] = model

test_data = grouped_data[(grouped_data['Year'] == 2021) & (grouped_data['Pos'].str.contains('MF', case=False))]
X_test = test_data[features]

# Make predictions using the trained models and calculate a score
for metric in metrics:
    test_data[f'Predicted_{metric}'] = models[metric].predict(X_test)

test_data['Score'] = test_data['Predicted_GI'] + test_data['Predicted_PrgC'] + test_data['Predicted_PrgP'] + test_data['Predicted_PrgR']

# Sort by score and display the top 20 players
top_performers = test_data.sort_values(by='Score', ascending=False).head(20)
print(top_performers[['Player', 'Nation', 'Squad', 'Age', 'Born', 'Year', 'Predicted_GI', 'Predicted_PrgC', 'Predicted_PrgP', 'Predicted_PrgR', 'Score']])
print()

<ipython-input-15-5601e905951d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[metric] = train_data['Gls'] + train_data['Ast'] if metric == 'GI' else train_data[metric]
<ipython-input-15-5601e905951d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[metric] = train_data['Gls'] + train_data['Ast'] if metric == 'GI' else train_data[metric]
<ipython-input-15-5601e905951d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

                    Player   Nation            Squad  Age  Born  Year  \
1815           bukayo saka  eng eng          arsenal   19  2001  2021   
1580        bernardo silva   pt por  manchester city   26  1994  2021   
1499   benjamin bourigeaud   fr fra           rennes   27  1994  2021   
4344       gerard deulofeu   es esp          udinese   27  1994  2021   
8570     matte9o guendouzi   fr fra        marseille   22  1999  2021   
4430      gianluca caprari   it ita    hellas verona   28  1993  2021   
9518       nicolf2 barella   it ita            inter   24  1997  2021   
8423           mason mount  eng eng          chelsea   22  1999  2021   
7327          lionel messi   ar arg         paris sg   34  1987  2021   
1122      antonio candreva   it ita        sampdoria   34  1987  2021   
11510          seko fofana   ci civ             lens   26  1995  2021   
2199    christopher nkunku   fr fra       rb leipzig   23  1997  2021   
9281           nabil fekir   fr fra            beti

<ipython-input-15-5601e905951d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[f'Predicted_{metric}'] = models[metric].predict(X_test)
<ipython-input-15-5601e905951d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[f'Predicted_{metric}'] = models[metric].predict(X_test)
<ipython-input-15-5601e905951d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

More refined for finding young players that would be good to purchase due to their high score across the multiple datapoints

In [ ]:
young_prospects = top_performers[top_performers['Age'] < 25]
young_prospects = young_prospects.sort_values(by='Score', ascending=False)
print(young_prospects.head(20))

                   Player   Pos   Nation  Age  Born  Year         Squad  \
1815          bukayo saka  FWMF  eng eng   19  2001  2021       arsenal   
8570    matte9o guendouzi    MF   fr fra   22  1999  2021     marseille   
9518      nicolf2 barella    MF   it ita   24  1997  2021         inter   
8423          mason mount    MF  eng eng   22  1999  2021       chelsea   
2199   christopher nkunku  FWMF   fr fra   23  1997  2021    rb leipzig   
9188         moussa diaby  FWMF   fr fra   22  1999  2021    leverkusen   
10609            raphinha  FWMF   br bra   24  1996  2021  leeds united   
4115       franck honorat  FWMF   fr fra   24  1996  2021         brest   

                     Comp  MP  Starts   Min   90s  Gls  Ast  G+A  G-PK  PK  \
1815   eng premier league  38      36  2978  33.1   11    7   18     9   2   
8570           fr ligue 1  38      35  3152  35.0    4    6   10     4   0   
9518           it serie a  36      36  2913  32.4    3   11   14     3   0   
8423   eng p